# Lab 02a: ISS decoding 

## Using starfish pixel spot decoder

Once the data is correctly loaded and processed, we are in the position of decoding the actual ISS spots. To this end, we will use again **starfish**, find the original tutorial [here](https://spacetx-starfish.readthedocs.io/en/mcai-api-additions/gallery/tutorials/pixelbased_decoding.html#sphx-glr-gallery-tutorials-pixelbased-decoding-py):

> Pixel-based decoding is the approach of localizing and decoding<sup>1</sup> molecules (e.g. RNA transcripts or rolonies) that does not rely on algorithms to find spots by fitting Gaussian profiles or local intensity maxima. Instead of finding spots to be decoded, it decodes every pixel and then connects potential pixels with the same codeword<sup>2</sup> from the codebook<sup>3</sup>  into spots. The strength of this approach is it works on dense data and noisy data where spot finding algorithms have a hard time accurately detecting spots. The weakness is that it is prone to false positives by decoding noise that would normally be ignored by spot finding algorithms.

[1]: Matching putative barcodes to codewords in a codebook to read out the corresponding target believed to be associated with that barcode. 

[2]: A codeword maps expected intensities across multiple image tiles within a field of view to the target that is encoded by the codeword. 

[3]: A codebook contains all the codewords needed by an experiment to decode an IntensityTable. It also contains a mapping of channels to the integer indices that are used by starfish to represent them internally.

We prepared a bigger tile by cropping one of the stitched parts of the raw data. 

In [ ]:
import numpy as np
import os
from starfish import Experiment
from starfish.image import Filter
from starfish.spots import DetectPixels
from starfish.types import Features

In [ ]:
exp = Experiment.from_json(
    os.path.join('../data/in_situ_sequencing/SpaceTX', "experiment.json")
)
imgs = exp['fov_001'].get_image('primary')

Following the tutorial, first we will apply high and low pass filters, designed to smooth the data before detecting the spots.

In [ ]:
ghp = Filter.GaussianHighPass(sigma=4)
glp = Filter.GaussianLowPass(sigma=.5)
ghp.run(imgs, in_place=True)
glp.run(imgs, in_place=True)

We can already use some kind of decoding to detect the expressed genes. In this case, we will use a pixel spot decoder. This can yield suboptimal results in terms of detection, but we will still use if for simiplicity sake. There are other approaches for performing this such as [bardensr](https://github.com/jacksonloper/bardensr) or [ISTDECO](https://github.com/axanderssonuu/istdeco) that allow decoding with a better performance. 

There are some hyperparameters that need to be tuned, but the most important input to the function is the **codebook** that contain which combination of rounds and channels (barcode) is translated to a specific gene.


In [ ]:
psd = DetectPixels.PixelSpotDecoder(
    codebook=exp.codebook,
    metric='euclidean',             # distance metric to use for computing distance between a pixel vector and a codeword
    norm_order=2,                   # the L_n norm is taken of each pixel vector and codeword before computing the distance. this is n
    distance_threshold=0.5,         # minimum distance between a pixel vector and a codeword for it to be called as a gene
    magnitude_threshold=1.77e-5,    # discard any pixel vectors below this magnitude
    min_area=1,                     # do not call a 'spot' if it's area is below this threshold (measured in pixels)
    max_area=np.inf,                # do not call a 'spot' if it's area is above this threshold (measured in pixels)
)
initial_spot_intensities, prop_results = psd.run(imgs, n_processes=None)

These approaches usually yield too many false positives, so it is a good dea to threshold based on random codes included for this purpose.

In [ ]:
# filter spots that do not pass thresholds
spot_intensities = initial_spot_intensities.loc[initial_spot_intensities[Features.PASSES_THRESHOLDS]]

This is how the decoded intensity table looks like:

In [ ]:
spot_intensities

Let's save it before continuing with the quality control using TissUUmaps. Starfish has a method for this:

In [ ]:
spot_intensities.to_netcdf('../results/in_situ_sequencing/spot_intensities.netcdf')

# Lab 02b: ISS decoding quality control

For this last step, we will use the TissUUmaps [Spot Inspector](https://tissuumaps.github.io/tutorials/#spot_inspector) plugin to visually assess the quality of the decoding. This plugin allows to explore raw in situ sequencing data by visualizing a grid of rounds and channels and drawing the trace of the codeword that the decoding algorithm decided to assign to a particular spot. Thus, in a visual and intuitive way, one may see if the decoding corresponds to what the raw data and potentially detect some error sources.

To begin, we will load the spot intensities saved in the previous part of the lab.

In [ ]:
from starfish import IntensityTable
from iss_utils import starfish2tmap

In [ ]:
spot_intensities = IntensityTable.open_netcdf('../results/in_situ_sequencing/spot_intensities.netcdf')

We developed a series of helper functions that will allow adapting the starfish files for opening in TissUUmaps. For more information, visit [the website](https://tissuumaps.github.io/tutorials/#starfish).

First, we will create a CSV file from a starfish experiments compatible with the TissUUmaps Spot Insepector plugin.

In [ ]:
csv_name = starfish2tmap.qc_csv(experiment=exp,
                     spot_intensities=spot_intensities,
                     output_path='../results/in_situ_sequencing')


Now, we can create the images from a starfish experiments to show for the Spot Inspector plugin.

In [ ]:
image_names = starfish2tmap.qc_images(imgs, exp, output_path='../results/in_situ_sequencing')

Finally, we can open TissUUmaps inside this notebook and play around with the plugin. Find some good and bad examples of decodings and try to think what went wrong with a particular spot.

In [ ]:
import tissuumaps.jupyter as tm

from urllib import request
os.makedirs(os.path.join(os.path.expanduser("~"), ".tissuumaps", "plugins"), exist_ok = True)
for ext in [".py",".js",".yml"]:
    request.urlretrieve(
        "https://tissuumaps.github.io/TissUUmaps/plugins/Spot_Inspector" + ext,
        os.path.join(os.path.expanduser("~"), ".tissuumaps", "plugins", 'Spot_Inspector' + ext)
    )

In [ ]:
tm.loaddata(images=image_names,csvFiles= csv_name, plugins=["Spot_Inspector"])